<a href="https://colab.research.google.com/github/aarnavg54/Deep-Learning-Radiomic-Stability/blob/main/FPN_with_DenseNet161.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing neccessary libraries
!pip install -U segmentation-models-pytorch
!pip install -U git+https://github.com/qubvel-org/segmentation_models.pytorch

  Cloning https://github.com/qubvel-org/segmentation_models.pytorch to /tmp/pip-req-build-1nut263w
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel-org/segmentation_models.pytorch /tmp/pip-req-build-1nut263w
  Resolved https://github.com/qubvel-org/segmentation_models.pytorch to commit e76ed01d49ff986be065fcc3c764255ce512cf16
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from torch.utils.data import Dataset

# This function creates a pytorch dataset
class HistologyDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype("float32")
        mask = self.masks[idx].astype("float32")
        image = torch.from_numpy(image).permute(2, 0, 1)
        mask = torch.from_numpy(mask).permute(2, 0, 1)
        return image, mask


In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Our preprocessed data

import numpy as np
X_train = np.load('/content/drive/MyDrive/X_train_ultrasound_images_256_2.npy')
y_train = np.load('/content/drive/MyDrive/y_train_ultrasound_images_256_2.npy')
X_val = np.load('/content/drive/MyDrive/X_val_ultrasound_images_256_2.npy')
y_val = np.load('/content/drive/MyDrive/y_val_ultrasound_images_256_2.npy')
X_test = np.load('/content/drive/MyDrive/X_test_ultrasound_images_256_2.npy')
y_test = np.load('/content/drive/MyDrive/y_test_ultrasound_images_256_2.npy')

# Pytorch training/testing/validation
train_dataset = HistologyDataset(X_train, y_train)
val_dataset = HistologyDataset(X_val, y_val)
test_dataset = HistologyDataset(X_test, y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Sanity check
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(1028, 256, 256, 3)
(1028, 256, 256, 1)
(220, 256, 256, 3)
(220, 256, 256, 1)
(221, 256, 256, 3)
(221, 256, 256, 1)


In [ ]:
# Importing our model from Qubvel's Segmentation Models GitHub repo
import segmentation_models_pytorch as smp

model = smp.FPN(
    encoder_name="densenet161",   # UPDATED
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/116M [00:00<?, ?B/s]

In [ ]:
# Dice loss
from segmentation_models_pytorch.losses import DiceLoss
import torch.optim as optim

dice_loss = DiceLoss(mode='binary', from_logits=True, smooth=1e-5)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Setting up the training loop
from torch.utils.data import DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Loading our training/ validation/ testing data
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # UPDATED
val_loader = DataLoader(val_dataset, batch_size=4)
test_loader = DataLoader(test_dataset)

In [ ]:
# Initializing the number of epochs
num_epochs = 500
# Early stopping settings
patience = 10
best_val_loss = float('inf')
epochs_no_improve = 0

checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.pth")

In [ ]:
# Looping through all 500 epochs
for epoch in range(1, num_epochs + 1):
    model.train()
    train_losses = []
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = dice_loss(outputs, masks)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    val_losses = []
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            loss = dice_loss(outputs, masks)
            val_losses.append(loss.item())

    avg_val_loss = sum(val_losses) / len(val_losses)
    # Printing as training/ validation loss as the model trains

    print(f"Epoch {epoch:03d}: Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Early stopping conditions
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Best model saved with Val Loss: {best_val_loss:.4f}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    # If the model's validation loss doesn't improve after 10 epochs, the model will automatically trigger early stopping

Epoch 001: Train Loss: 0.1714 | Val Loss: 0.1340
Best model saved with Val Loss: 0.1340
Epoch 002: Train Loss: 0.0983 | Val Loss: 0.1188
Best model saved with Val Loss: 0.1188
Epoch 003: Train Loss: 0.0781 | Val Loss: 0.1064
Best model saved with Val Loss: 0.1064
Epoch 004: Train Loss: 0.0675 | Val Loss: 0.1080
Epoch 005: Train Loss: 0.0566 | Val Loss: 0.0957
Best model saved with Val Loss: 0.0957
Epoch 006: Train Loss: 0.0497 | Val Loss: 0.0930
Best model saved with Val Loss: 0.0930
Epoch 007: Train Loss: 0.0455 | Val Loss: 0.0915
Best model saved with Val Loss: 0.0915
Epoch 008: Train Loss: 0.0422 | Val Loss: 0.0888
Best model saved with Val Loss: 0.0888
Epoch 009: Train Loss: 0.0399 | Val Loss: 0.0930
Epoch 010: Train Loss: 0.0375 | Val Loss: 0.0900
Epoch 011: Train Loss: 0.0353 | Val Loss: 0.0882
Best model saved with Val Loss: 0.0882
Epoch 012: Train Loss: 0.0338 | Val Loss: 0.0829
Best model saved with Val Loss: 0.0829
Epoch 013: Train Loss: 0.0329 | Val Loss: 0.0900
Epoch 014: T

In [ ]:
from scipy.spatial.distance import directed_hausdorff

def hausdorff_distance(pred, target):
    pred = pred.squeeze().cpu().numpy()
    target = target.squeeze().cpu().numpy()

    # Get coordinates of boundary pixels
    pred_coords = np.argwhere(pred > 0.5)
    target_coords = np.argwhere(target > 0.5)

    # Handle empty masks
    if len(pred_coords) == 0 or len(target_coords) == 0:
        return np.nan  # Return NaN if either mask is empty

    # Compute both directions
    hd1 = directed_hausdorff(pred_coords, target_coords)[0]
    hd2 = directed_hausdorff(target_coords, pred_coords)[0]

    return max(hd1, hd2)

In [ ]:
def calculate_metrics(preds, targets, smooth=1e-6):
    # Flatten tensors and calculate TP, FP, FN, TN
    preds_flat = preds.flatten(1)
    targets_flat = targets.flatten(1)

    tp = (preds_flat * targets_flat).sum(1)
    fp = (preds_flat * (1 - targets_flat)).sum(1)
    fn = ((1 - preds_flat) * targets_flat).sum(1)
    tn = ((1 - preds_flat) * (1 - targets_flat)).sum(1)

    # Standard metrics
    metrics = {
        'Dice': ((2 * tp + smooth) / (tp + fp + tp + fn + smooth)).mean().item(),
        'IoU': ((tp + smooth) / (tp + fp + fn + smooth)).mean().item(),
        'Precision': ((tp + smooth) / (tp + fp + smooth)).mean().item(),
        'Recall': ((tp + smooth) / (tp + fn + smooth)).mean().item(),
        'Specificity': ((tn + smooth) / (tn + fp + smooth)).mean().item(),
        'Accuracy': ((tp + tn + smooth) / (tp + tn + fp + fn + smooth)).mean().item(),
    }

    # Hausdorff Distance (handle batch)
    hd_values = []
    for p, t in zip(preds, targets):
        hd = hausdorff_distance(p, t)
        if not np.isnan(hd):
            hd_values.append(hd)
    metrics['HD95'] = np.percentile(hd_values, 95) if hd_values else np.nan

    # Tumor size error
    pred_area = preds_flat.sum(1)
    target_area = targets_flat.sum(1)
    metrics['Size_Error'] = ((pred_area - target_area).abs() / (target_area + smooth)).mean().item()

    return metrics

In [ ]:
model.eval()
all_metrics = []

with torch.no_grad():
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        metrics = calculate_metrics(preds, masks)
        all_metrics.append(metrics)

# Aggregate results (ignoring NaN values)
final_metrics = {
    k: np.nanmean([m[k] for m in all_metrics if not np.isnan(m[k])])
    for k in all_metrics[0].keys()
}


In [ ]:
print("Final Metrics:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value:.4f}")

Final Metrics:
Dice: 0.9280
IoU: 0.8710
Precision: 0.9393
Recall: 0.9223
Specificity: 0.9890
Accuracy: 0.9780
HD95: 13.5048
Size_Error: 0.0752


In [ ]:
from scipy import stats

def parametric_ci(metric_values, ci=95):
    """Calculate parametric CI assuming normal distribution"""
    mean = np.nanmean(metric_values)
    sem = stats.sem(metric_values, nan_policy='omit')  # Standard error
    ci_val = sem * stats.t.ppf((1 + ci/100) / 2, len(metric_values)-1)
    return mean - ci_val, mean + ci_val

# Usage
iou_scores = [m['IoU'] for m in all_metrics]
iou_ci = parametric_ci(iou_scores, ci=95)
print(f"IoU: {np.mean(iou_scores):.3f} ({iou_ci[0]:.3f}-{iou_ci[1]:.3f})")


IoU: 0.871 (0.858-0.884)


In [ ]:
def evaluate_with_ci(model, test_loader, device, n_bootstraps=1000):
    model.eval()
    all_metrics = []

    with torch.no_grad():
        for images, masks in test_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            metrics = calculate_metrics(preds, masks)
            all_metrics.append(metrics)

    # Aggregate metrics
    metric_names = all_metrics[0].keys()
    results = {}

    for name in metric_names:
        values = [m[name] for m in all_metrics]

        # Use bootstrapping for Dice/IoU/HD95
        if name in ['Dice', 'IoU', 'HD95']:
            ci = bootstrap_ci(values, n_bootstraps=n_bootstraps)
        else:
            ci = parametric_ci(values)

        results[name] = {
            'mean': np.nanmean(values),
            'ci_lower': ci[0],
            'ci_upper': ci[1],
            'ci_method': 'bootstrap' if name in ['Dice', 'IoU', 'HD95'] else 'parametric'
        }

    return results


In [ ]:
checkpoint_path = '/content/drive/MyDrive/FPN_densenet161_0.9280.pth'

In [ ]:
torch.save(model.state_dict(), checkpoint_path)
print(f"Model saved to {checkpoint_path}")

Model saved to /content/drive/MyDrive/FPN_densenet161_0.9280.pth
